In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from sklearn.metrics import jaccard_score, f1_score
import random
import matplotlib.pyplot as plt

# Set up constants
IMG_HEIGHT, IMG_WIDTH = 64, 64
BATCH_SIZE = 16
EPOCHS = 5
IMAGE_DIR = "../datasets/dataset2/1/face_crop"
MASK_DIR = "../datasets/dataset2/1/face_crop_segmentation"
RESULTS_DIR = "./results"
TRAIN_SIZE = 1000
TEST_SIZE = 100
os.makedirs(RESULTS_DIR, exist_ok=True)

# Load and preprocess dataset
def load_data(image_dir, mask_dir, img_size=(IMG_HEIGHT, IMG_WIDTH), limit=TRAIN_SIZE):
    image_files = sorted(os.listdir(image_dir))
    mask_files = sorted(os.listdir(mask_dir))
    selected_indices = random.sample(range(len(image_files)), limit)
    images, masks = [], []
    for i in selected_indices:
        img = cv2.imread(os.path.join(image_dir, image_files[i]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size).astype(np.float32) / 255.0
        mask = cv2.imread(os.path.join(mask_dir, mask_files[i]), cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, img_size).astype(np.float32) / 255.0
        mask = mask.reshape(img_size[0], img_size[1], 1)
        images.append(img)
        masks.append(mask)
    
    return np.array(images), np.array(masks)

# Load training and testing data
X_train, Y_train = load_data(IMAGE_DIR, MASK_DIR, limit=TRAIN_SIZE)
X_test, Y_test = load_data(IMAGE_DIR, MASK_DIR, limit=TEST_SIZE)

print(f"Training set: {X_train.shape}, {Y_train.shape}")
print(f"Test set: {X_test.shape}, {Y_test.shape}")

# Create TensorFlow dataset
def create_dataset(X, Y):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.shuffle(len(X)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return dataset

train_dataset = create_dataset(X_train, Y_train)

# U-Net Model
def unet_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)):
    inputs = tf.keras.Input(shape=input_shape)

    def conv_block(x, filters):
        x = tf.keras.layers.Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
        x = tf.keras.layers.Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
        return x

    def upsample_block(x, skip, filters):
        x = tf.keras.layers.Conv2DTranspose(filters, (2, 2), strides=(2, 2), padding='same')(x)
        x = tf.keras.layers.Concatenate()([x, skip])
        return conv_block(x, filters)

    f = [64, 128, 256, 512, 1024]
    c1 = conv_block(inputs, f[0])
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)
    c2 = conv_block(p1, f[1])
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
    c3 = conv_block(p2, f[2])
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
    c4 = conv_block(p3, f[3])
    p4 = tf.keras.layers.MaxPooling2D((2, 2))(c4)
    bottleneck = conv_block(p4, f[4])
    u4 = upsample_block(bottleneck, c4, f[3])
    u3 = upsample_block(u4, c3, f[2])
    u2 = upsample_block(u3, c2, f[1])
    u1 = upsample_block(u2, c1, f[0])
    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='tanh')(u1)
    model = tf.keras.Model(inputs, outputs)
    return model

# Compile and train model
model = unet_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss='binary_crossentropy')

# Training loop (no validation metrics per epoch)
for epoch in range(EPOCHS):
    history = model.fit(train_dataset, epochs=1, verbose=1)
    y_train_pred = model.predict(X_train)
    # Compute training IoU and Dice scores
    y_train_pred_bin = (y_train_pred.flatten() > 0.5).astype(np.uint8)
    y_train_true_bin = (Y_train.flatten() > 0.5).astype(np.uint8)
    train_iou = jaccard_score(y_train_true_bin, y_train_pred_bin, average='binary')
    train_dice = f1_score(y_train_true_bin, y_train_pred_bin, average='binary')
    
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {history.history['loss'][-1]:.4f} - Train IoU: {train_iou:.4f} - Train Dice: {train_dice:.4f}")

# Final testing on 100 images
y_test_pred = model.predict(X_test)

# Compute IoU and Dice scores for test set
ious, dices = [], []

for i in range(TEST_SIZE):
    y_true_bin = (Y_test[i].flatten() > 0.5).astype(np.uint8)
    y_pred_bin = (y_test_pred[i].flatten() > 0.5).astype(np.uint8)
    
    ious.append(jaccard_score(y_true_bin, y_pred_bin, average='binary'))
    dices.append(f1_score(y_true_bin, y_pred_bin, average='binary'))

# Print final evaluation metrics
mean_iou = np.mean(ious)
mean_dice = np.mean(dices)

print(f"\nFinal Test Results on {TEST_SIZE} images:")
print(f"Mean IoU: {mean_iou:.4f}")
print(f"Mean Dice Score: {mean_dice:.4f}")

# Save & visualize 5 random test predictions
sample_indices = random.sample(range(TEST_SIZE), 5)

for i, idx in enumerate(sample_indices):
    img = X_test[idx]
    mask = Y_test[idx].squeeze()
    pred_mask = y_test_pred[idx].squeeze()
    pred_mask = (pred_mask > 0.5).astype(np.uint8)

    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.title("Input Image")
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.imshow(mask, cmap="gray")
    plt.title("Ground Truth Mask")
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(pred_mask, cmap="gray")
    plt.title("Predicted Mask")
    plt.axis("off")

    result_path = os.path.join(RESULTS_DIR, f"test_result_{i+1}.png")
    plt.savefig(result_path)
    plt.close()

print(f"Visualization results saved to {RESULTS_DIR}")